# Day 11: Seating System

https://adventofcode.com/2020/day/11

## Part 1

Now we'll be dealing with some 2-D grid action. Fun times!

Rules state that each round of seating applies these rules *simultaneously*:

- If a seat is empty (`L`) and there are no occupied seats adjacent to it, the seat becomes occupied.
- If a seat is occupied (`#`) and four or more seats adjacent to it are also occupied, the seat becomes empty.
- Otherwise, the seat's state does not change.

To apply these simultaneously as described, we'll generate a new layout for the round, then add in the new seat assignments as we go, eventually returning the full new layout.

In [80]:
# As always, grab that input!
from pathlib import Path
from typing import List, Callable

INPUTS = Path('input.txt').resolve().read_text().strip().split('\n')

FLOOR = "."
EMPTY_SEAT = "L"
OCCUPIED_SEAT = "#"

starting_layout = INPUTS

def num_adjacent_occupied(layout, row, col) -> int:
    """Helper method that checks for the number of adjacent occupied seats
    to a specific location, given the `row` and `col` coordinates of that location.
    """
    num_occupied = 0

    # We'll use ranges for this iteration, starting from the previous row (row-1)
    # and ending on two rows ahead (row+2). This is due to `range` being exclusive
    # for the endpoint.
    # Ex: if x = 2, range(x-1, x+2) == range(1, 4) == [1, 2, 3]
    # This provides us the correct indices to access.
    # Additionally, we wrap `x-1` in `max()` with that and `0` as arguments.
    # This provents a 0 index from wrapping around to -1, which is a valid index
    # for the LAST element in the set, but is not one we wish to compare here!
    for idx in range(max(row-1, 0), row+2):
        for idy in range(max(col-1, 0), col+2):
            if idx == row and idy == col:
                # This is the current seat, so skip it.
                continue
            try:
                if layout[idx][idy] == OCCUPIED_SEAT:
                    num_occupied += 1
            except IndexError:
                # No seat here (end of the list or string reached).
                # Skip it, move on.
                continue
    return num_occupied

def seating_round(layout: List[str], adjacency_func: Callable = num_adjacent_occupied, adjacency_tolerance: int = 4) -> List[str]:
    """Applies one round of rules to a seating layout, returning
    a copy of that layout with the rules applied for a single round.
    """

    new_layout = []
    for row_idx, row in enumerate(layout):
        new_row = ""
        for col_idx, space in enumerate(row):
            new_space = space
            adjacent_occupied = adjacency_func(layout, row_idx, col_idx)
            if space == EMPTY_SEAT and not adjacent_occupied:
                new_space = OCCUPIED_SEAT
            elif space == OCCUPIED_SEAT and adjacent_occupied >= adjacency_tolerance:
                new_space = EMPTY_SEAT
            new_row += new_space
        new_layout.append(new_row)
    
    return new_layout

In [81]:
# Sanity checks

sanity = (
    "L.LL.LL.LL\n"
    "LLLLLLL.LL\n"
    "L.L.L..L..\n"
    "LLLL.LL.LL\n"
    "L.LL.LL.LL\n"
    "L.LLLLL.LL\n"
    "..L.L.....\n"
    "LLLLLLLLLL\n"
    "L.LLLLLL.L\n"
    "L.LLLLL.LL"
).strip().split('\n')

expecteds = [
    (
        "#.##.##.##\n"
        "#######.##\n"
        "#.#.#..#..\n"
        "####.##.##\n"
        "#.##.##.##\n"
        "#.#####.##\n"
        "..#.#.....\n"
        "##########\n"
        "#.######.#\n"
        "#.#####.##"
    ).strip().split('\n'),
    (
        "#.LL.L#.##\n"
        "#LLLLLL.L#\n"
        "L.L.L..L..\n"
        "#LLL.LL.L#\n"
        "#.LL.LL.LL\n"
        "#.LLLL#.##\n"
        "..L.L.....\n"
        "#LLLLLLLL#\n"
        "#.LLLLLL.L\n"
        "#.#LLLL.##"
    ).strip().split('\n'),
    (
        "#.##.L#.##\n"
        "#L###LL.L#\n"
        "L.#.#..#..\n"
        "#L##.##.L#\n"
        "#.##.LL.LL\n"
        "#.###L#.##\n"
        "..#.#.....\n"
        "#L######L#\n"
        "#.LL###L.L\n"
        "#.#L###.##"
    ).strip().split('\n'),
]

round1 = seating_round(sanity)
assert round1 == expecteds[0]

round2 = seating_round(round1)
assert round2 == expecteds[1]

round3 = seating_round(round2)
assert round3 == expecteds[2]

Sanity checks look good, let's continue.

To find the stable layout in which no changes are made, we need to apply a `seating_round` to our layout and then compare the contents of the new and old layout to see if they match. If they do, we exit; if they don't, we run a new round until some stable round is reached.

In [82]:
from typing import Tuple

def find_stable_layout(layout: List[str], adjacency_func: Callable = num_adjacent_occupied, adjacency_tolerance: int = 4) -> Tuple[List[str], int]:
    # Initially we can just run a round on the starting layout.
    old_layout = layout
    new_layout = seating_round(old_layout, adjacency_func=adjacency_func, adjacency_tolerance=adjacency_tolerance)
    roundnum = 1
    # Then we can start iterating.
    while new_layout != old_layout:
        old_layout = new_layout
        new_layout = seating_round(old_layout, adjacency_func=adjacency_func, adjacency_tolerance=adjacency_tolerance)
        roundnum += 1
    return new_layout, roundnum

stable_layout, num_rounds = find_stable_layout(starting_layout)

print(f"Stable layout achieved in {num_rounds} seating rounds.")

num_occupied_at_stable = sum([1 for row in stable_layout for seat in row if seat == OCCUPIED_SEAT])
print(f"There are {num_occupied_at_stable} occupied seats in the stable layout.")

Stable layout achieved in 98 seating rounds.
There are 2334 occupied seats in the stable layout.


## Part 2

Seems a bit tricky at first, but not terribly. Only a few changes are needed above to accommodate:

- Originally I had `num_adjacent_occupied` as a helper function defined inside `seating_round`, but have now moved it outside that function and given it the `layout` argument it didn't need before.
- `seating_round` now takes two new arguments, `adjacency_func` and `adjacency_tolerance`, with defaults to the original problem's requirements (`num_adjacent_occupied` and `4`, respectively).
- And other code in `seating_round` as well as `find_stable_layout` adjusted to accommodate those changes.

So, now we should be able to re-use `seating_round` by supplying a new adjacency function to return the number of occupied seats. This time it will crawl out from the current seat in eight directions, looking for:

1. A floor space, allowing it to continue;
2. A wall (where some index crosses below 0 or above the length of the rows or cols), allowing it to stop;
3. A seat space, occupied or not, allowing it to stop and return its result.

Finally, we should just be able to give the new tolerance, `5`, to `seating_round` to get the correct new layout.

In [83]:
def num_adjacent_cardinal_seats(layout, row, col) -> int:
    """Find the number of "adjacent" seats according to rules in Part 2,
    which essentially use cardinal directions.
    """
    def is_occupied_in_direction(delta_row: int, delta_col: int) -> bool:
        """Helper method, determines if there's an occupied seat
        in a given direction, defined by a delta change in rol and col indices.
        """
        nonlocal layout, row, col
        if not delta_row and not delta_col:
            raise ValueError("Can't move nowhere!")

        curr_row, curr_col = row, col
        while True:
            curr_row += delta_row
            curr_col += delta_col
            if curr_row < 0 or curr_row >= len(layout) or curr_col < 0 or curr_col >= len(layout[curr_row]):
                # Hit a wall, no further spaces to check.
                # Nothing is occupied in this direction if we got this far.
                return False
            if layout[curr_row][curr_col] == OCCUPIED_SEAT:
                # Seat located, and it's occupied.
                return True
            if layout[curr_row][curr_col] == EMPTY_SEAT:
                # Seat located, and it's empty.
                return False
            # No seat found, no wall hit yet. Continue naturally.
            # Unlikely we won't hit one of the above conditions

    occupied_seats = 0
    for delta_row in [-1, 0, 1]:
        for delta_col in [-1, 0, 1]:
            if not delta_row and not delta_col:
                # Center point. Skip
                continue
            if is_occupied_in_direction(delta_row, delta_col):
                occupied_seats += 1
    return occupied_seats

In [84]:
# Sanity checks: num_adjacent

cardinal_sanity1 = (
    ".......#.\n"
    "...#.....\n"
    ".#.......\n"
    ".........\n"
    "..#L....#\n"
    "....#....\n"
    ".........\n"
    "#........\n"
    "...#....."
).strip().split('\n')

assert num_adjacent_cardinal_seats(cardinal_sanity1, 4, 3) == 8

cardinal_sanity2 = (
    ".............\n"
    ".L.L.#.#.#.#.\n"
    "............."
).strip().split('\n')

assert num_adjacent_cardinal_seats(cardinal_sanity2, 1, 1) == 0
assert num_adjacent_cardinal_seats(cardinal_sanity2, 1, 3) == 1

cardinal_sanity3 = (
    ".##.##.\n"
    "#.#.#.#\n"
    "##...##\n"
    "...L...\n"
    "##...##\n"
    "#.#.#.#\n"
    ".##.##."
).strip().split('\n')

assert num_adjacent_cardinal_seats(cardinal_sanity3, 3, 3) == 0

In [85]:
# Sanity checks: new rounds of seating:

sanity_cardinal_starting = (
    "L.LL.LL.LL\n"
    "LLLLLLL.LL\n"
    "L.L.L..L..\n"
    "LLLL.LL.LL\n"
    "L.LL.LL.LL\n"
    "L.LLLLL.LL\n"
    "..L.L.....\n"
    "LLLLLLLLLL\n"
    "L.LLLLLL.L\n"
    "L.LLLLL.LL"
).strip().split('\n')

expecteds = [
    (
        "#.##.##.##\n"
        "#######.##\n"
        "#.#.#..#..\n"
        "####.##.##\n"
        "#.##.##.##\n"
        "#.#####.##\n"
        "..#.#.....\n"
        "##########\n"
        "#.######.#\n"
        "#.#####.##"
    ).strip().split('\n'),
    (
        "#.LL.LL.L#\n"
        "#LLLLLL.LL\n"
        "L.L.L..L..\n"
        "LLLL.LL.LL\n"
        "L.LL.LL.LL\n"
        "L.LLLLL.LL\n"
        "..L.L.....\n"
        "LLLLLLLLL#\n"
        "#.LLLLLL.L\n"
        "#.LLLLL.L#"
    ).strip().split('\n'),
    (
        "#.L#.##.L#\n"
        "#L#####.LL\n"
        "L.#.#..#..\n"
        "##L#.##.##\n"
        "#.##.#L.##\n"
        "#.#####.#L\n"
        "..#.#.....\n"
        "LLL####LL#\n"
        "#.L#####.L\n"
        "#.L####.L#"
    ).strip().split('\n'),
    (
        "#.L#.L#.L#\n"
        "#LLLLLL.LL\n"
        "L.L.L..#..\n"
        "##LL.LL.L#\n"
        "L.LL.LL.L#\n"
        "#.LLLLL.LL\n"
        "..L.L.....\n"
        "LLLLLLLLL#\n"
        "#.LLLLL#.L\n"
        "#.L#LL#.L#"
    ).strip().split('\n'),
    (
        "#.L#.L#.L#\n"
        "#LLLLLL.LL\n"
        "L.L.L..#..\n"
        "##L#.#L.L#\n"
        "L.L#.#L.L#\n"
        "#.L####.LL\n"
        "..#.#.....\n"
        "LLL###LLL#\n"
        "#.LLLLL#.L\n"
        "#.L#LL#.L#"
    ).strip().split('\n'),
    (
        "#.L#.L#.L#\n"
        "#LLLLLL.LL\n"
        "L.L.L..#..\n"
        "##L#.#L.L#\n"
        "L.L#.LL.L#\n"
        "#.LLLL#.LL\n"
        "..#.L.....\n"
        "LLL###LLL#\n"
        "#.LLLLL#.L\n"
        "#.L#LL#.L#"
    ).strip().split('\n'),
]

cardinal_round_0 = seating_round(sanity_cardinal_starting, adjacency_func=num_adjacent_cardinal_seats, adjacency_tolerance=5)
assert cardinal_round_0 == expecteds[0]

cardinal_round_1 = seating_round(cardinal_round_0, adjacency_func=num_adjacent_cardinal_seats, adjacency_tolerance=5)
assert cardinal_round_1 == expecteds[1]

cardinal_round_2 = seating_round(cardinal_round_1, adjacency_func=num_adjacent_cardinal_seats, adjacency_tolerance=5)
assert cardinal_round_2 == expecteds[2]

cardinal_round_3 = seating_round(cardinal_round_2, adjacency_func=num_adjacent_cardinal_seats, adjacency_tolerance=5)
assert cardinal_round_3 == expecteds[3]

cardinal_round_4 = seating_round(cardinal_round_3, adjacency_func=num_adjacent_cardinal_seats, adjacency_tolerance=5)
assert cardinal_round_4 == expecteds[4]

cardinal_round_5 = seating_round(cardinal_round_4, adjacency_func=num_adjacent_cardinal_seats, adjacency_tolerance=5)
assert cardinal_round_5 == expecteds[5]

# Final round should be stable and the same as previous
cardinal_round_6 = seating_round(cardinal_round_5, adjacency_func=num_adjacent_cardinal_seats, adjacency_tolerance=5)
assert cardinal_round_6 == expecteds[5]
assert cardinal_round_6 == cardinal_round_5

In [86]:
new_stable_layout, numrounds = find_stable_layout(starting_layout, adjacency_func=num_adjacent_cardinal_seats, adjacency_tolerance=5)

print(f"Stable layout in Part 2 rules reached in {numrounds} rounds.")
num_occupied_at_stable = sum([1 for row in new_stable_layout for seat in row if seat == OCCUPIED_SEAT])
print(f"There are {num_occupied_at_stable} occupied seats in this stable layout.")

Stable layout in Part 2 rules reached in 86 rounds.
There are 2100 occupied seats in this stable layout.
